In [2]:
import pandas as pd

In [3]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
df = df.iloc[:300]

In [13]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
answer_llm = df.iloc[0].answer_llm
answer_llm_embedding = embedding_model.encode(answer_llm)

In [27]:
answer_orig = df.iloc[0].answer_orig
answer_orig_embedding = embedding_model.encode(answer_orig)

In [15]:
answer_llm_embedding[0]

-0.42244655

In [30]:
answer_orig_embedding.dot(answer_llm_embedding)*1

17.515987396240234

In [44]:
import numpy as np

def compute_dot_product(original,llm):
    original_embeddings = embedding_model.encode(original)
    llm_embeddings = embedding_model.encode(llm)
    dot_product = original_embeddings.dot(llm_embeddings)
    return dot_product

In [45]:
df['answer_dot_product'] = np.vectorize(compute_dot_product)(df['answer_orig'],df['answer_llm'])

/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in compute_dot_product (vectorized)
  outputs = ufunc(*inputs)
/home/codespace/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in compute_dot_product (vectorized)
  outputs = ufunc(*inputs)


In [48]:
df['answer_dot_product'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: answer_dot_product, dtype: float64

In [53]:
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

def compute_cosine_product(original,llm):
    norm_original_embeddings = normalize_vector(embedding_model.encode(original))
    norm_llm_embeddings = normalize_vector(embedding_model.encode(llm))
    cosine_product = norm_original_embeddings.dot(norm_llm_embeddings)
    return cosine_product

In [54]:
df['answer_cosine_product'] = np.vectorize(compute_cosine_product)(df['answer_orig'],df['answer_llm'])

In [55]:
df['answer_cosine_product'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: answer_cosine_product, dtype: float64

In [56]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [61]:
r = df.iloc[10]

In [62]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [63]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [65]:
f_scores = [v['f'] for v in scores.values()]

In [67]:
average_f_score = sum(f_scores)/len(f_scores)
average_f_score

0.35490034990035496

In [72]:
score_dfs = []
for row in df:
    score = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    score_dfs.append(pd.DataFrame(scores))
scores_df = pd.concat(score_dfs)

TypeError: string indices must be integers